Realizado por Breyner Andreit Rincon, Cristian Ballen Gamboa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark


from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, sum, concat, col, udf
from pyspark.sql.types import DoubleType
import re

In [ ]:
# Función para limpiar y convertir administracion a número
def limpiar_administracion(administracion):
    # Patrón para extraer números (incluyendo puntos y comas)
    patron_numeros = re.compile(r'[\d\.,]+')

    # Buscar el primer número en el texto y extraerlo
    match = patron_numeros.search(administracion)
    if match:
        valor = match.group(0)
        # Remover comas y puntos para obtener el valor numérico correcto
        valor_limpio = valor.replace('.', '').replace(',', '')
        # Convertir a float
        valor_numerico = float(valor_limpio)
        return valor_numerico
    else:
        return None

# Registrar la función UDF
limpiar_admin_udf = udf(limpiar_administracion, DoubleType())

In [ ]:
spark = SparkSession.builder.master("local[1]").appName("housing_fincaraiz_resuelto").getOrCreate()

# Ruta donde está ubicado tu archivo CSV
ruta = "/content/drive/MyDrive/bigData/ajustado.csv"

In [ ]:


# Cargar el archivo CSV y delimitar por punto y coma cada celda
df_spark = spark.read.format("csv").option("header", "true").option("delimiter", ";").load(ruta)

# Aplicar los filtros necesarios
df_filter = df_spark.filter(col('estado') != 'No definida') \
                    .filter(col('administracion') != 'No definida') \
                    .filter(col('antiguedad') != 'No definida') \
                    .filter(col('habitaciones') != 'No definida') \
                    .filter(col('baños') != 'No definida') \
                    .filter(col('area_privada') != '0 m²')

# Renombrar las columnas problemáticas
df_renamed = df_filter.withColumnRenamed('Área construida', 'area_construida') \
                      .withColumnRenamed('Área privada', 'area_privada') \
                      .withColumnRenamed('Estado', 'estado') \
                      .withColumnRenamed('Antigüedad', 'antiguedad') \
                      .withColumnRenamed('Administración', 'administracion') \
                      .withColumnRenamed('Precio_m2', 'precio_m2') \
                      .withColumnRenamed('Ubicación', 'ubicacion') \
                      .withColumnRenamed('Habitaciones', 'habitaciones') \
                      .withColumnRenamed('Baños', 'baños') \
                      .withColumnRenamed('Parqueaderos', 'parqueaderos') \
                      .withColumnRenamed('Estrato', 'estrato')

# Seleccionar solo las columnas especificadas
df_selected = df_renamed.select(
    col('habitaciones'),
    col('baños'),
    col('parqueaderos'),
    col('area_construida'),
    col('area_privada'),
    col('estrato'),
    col('estado'),
    col('antiguedad'),
    col('administracion'),
    col('precio_m2'),
    col('ubicacion')
)

# Mostrar las primeras filas del DataFrame para verificar
df_selected.show(200)



+------------+-----+------------+---------------+------------+-------+----------+--------------+---------------+------------------+--------------------+
|habitaciones|baños|parqueaderos|area_construida|area_privada|estrato|    estado|    antiguedad| administracion|         precio_m2|           ubicacion|
+------------+-----+------------+---------------+------------+-------+----------+--------------+---------------+------------------+--------------------+
|           1|    1|           0|          31 m²|       31 m²|      4| Excelente| menor a 1 año|  $ 130.000 COP| $ 7.419.354,84*m²|             Mazuren|
|           3|    3|           1|         150 m²|      150 m²|      6|     Bueno|más de 30 años|  $ 872.000 COP| $ 6.533.333,33*m²|          La Cabrera|
|           4|    4|           2|         264 m²|      264 m²|      5|     Bueno|más de 30 años|  $ 836.000 COP|  $ 5.303.030,3*m²| NICOLÁS DE FEDERMÁN|
|           3|    3|           1|         140 m²|      140 m²|      4|     Bueno|m

In [ ]:
spark = SparkSession.builder.master("local[1]").appName("housing_fincaraiz_resuelto").getOrCreate()

# Ruta donde está ubicado tu archivo CSV
ruta = "/content/drive/MyDrive/bigData/ajustado.csv"

# Cargar el archivo CSV y delimitar por punto y coma cada celda
df_spark = spark.read.format("csv").option("header", "true").option("delimiter", ";").load(ruta)

# Aplicar los filtros necesarios
df_filter = df_spark.filter(col('estado') != 'No definida') \
                    .filter(col('administracion') != 'No definida') \
                    .filter(col('antiguedad') != 'No definida') \
                    .filter(col('habitaciones') != 'No definida') \
                    .filter(col('baños') != 'No definida')  \


# Renombrar las columnas problemáticas
df_renamed = df_filter.withColumnRenamed('Área construida', 'area_construida') \
                      .withColumnRenamed('Área privada', 'area_privada') \
                      .withColumnRenamed('Estado', 'estado') \
                      .withColumnRenamed('Antigüedad', 'antiguedad') \
                      .withColumnRenamed('Administración', 'administracion') \
                      .withColumnRenamed('Precio_m2', 'precio_m2') \
                      .withColumnRenamed('Ubicación', 'ubicacion') \
                      .withColumnRenamed('Habitaciones', 'habitaciones') \
                      .withColumnRenamed('Baños', 'baños') \
                      .withColumnRenamed('Parqueaderos', 'parqueaderos') \
                      .withColumnRenamed('Estrato', 'estrato')

# Seleccionar solo las columnas especificadas
df_selected = df_renamed.select(
    col('habitaciones'),
    col('baños'),
    col('parqueaderos'),
    col('area_construida'),
    col('area_privada'),
    col('estrato'),
    col('estado'),
    col('antiguedad'),
    col('administracion'),
    col('precio_m2'),
    col('ubicacion')
)

df_cleaned = df_selected.withColumn('valor administracion', limpiar_admin_udf(col('administracion')))

# Eliminar la columna original 'administracion' si ya no la necesitas
df_cleaned = df_cleaned.drop('administracion')

df_final = df_cleaned.filter(col('valor administracion') > 1000000 ) \
                      .filter(col('ubicacion') == "San José de Bavaria")

# Mostrar las primeras filas del DataFrame para verificar
df_final.show(1000)



+------------+-----+------------+---------------+------------+-------+---------+------------+-----------------+-------------------+--------------------+
|habitaciones|baños|parqueaderos|area_construida|area_privada|estrato|   estado|  antiguedad|        precio_m2|          ubicacion|valor administracion|
+------------+-----+------------+---------------+------------+-------+---------+------------+-----------------+-------------------+--------------------+
|           3|    5|           6|         390 m²|      800 m²|      5|Excelente|  1 a 8 años|$ 6.410.256,41*m²|San José de Bavaria|           1050000.0|
|           5|    7|           4|         570 m²|        0 m²|      5|Excelente| 9 a 15 años|$ 5.087.017,54*m²|San José de Bavaria|           1200000.0|
|           3|    5|           6|         390 m²|      800 m²|      5|Excelente|  1 a 8 años|$ 6.410.256,41*m²|San José de Bavaria|           1050000.0|
|           5|    7|           4|         570 m²|        0 m²|      5|Excelente| 9